In [18]:
import serial

import time
import csv
import matplotlib
matplotlib.use("tkAgg")
import matplotlib.pyplot as plt
import numpy as np

import re
import pandas as pd
import dask.dataframe as dd
import streamz.dataframe as sdf
# from streamz.dataframe import StreamingDataFrame

# import threading, queue
import multiprocess as mp
# from multiprocessing import Process, Pipe
import signal
import os
from subprocess import check_output

from scipy.ndimage import gaussian_filter1d

In [3]:
portAddr = "/dev/cu.usbmodem14101"
baud = 115200

In [4]:
ser = serial.Serial(
    port=portAddr,
    baudrate=baud,
    bytesize=8,
    timeout=2,
    stopbits=serial.STOPBITS_ONE
)
ser.flushInput()



This just is a utility fn to extract values out of string we get from usb

In [5]:
def process_serial_out(ascii_string):
    ascii_string += ','
    findall = re.findall(r'(?P<var>.*?)=(?P<out>.*?),', ascii_string)
    extracted = {k.strip().lower(): float(v.strip()) for k,v in findall}
    return extracted

This is the first worker. USB -> arduino_worker -> usb_reading_queue

In [6]:

def arduino_worker(queue):
    print("worker starting...")
    while True:
        try:
            ser_bytes = ser.readline()
            try:
                print("got", ser_bytes)
                decoded_bytes = ser_bytes.decode('ascii')
                try:
                    row = process_serial_out(decoded_bytes)
                except:
                    continue
                print(decoded_bytes)
                queue.put(row)
            except:
                print_exc()
                print("exception reading one line from arduino")
                continue
        except:
            print_exc()
            print("Keyboard Interrupt")
            break

Prathik's code, with small modifications that might not be smart

In [7]:
## NOTE!! MKIMMINS: this now has accel z smooth in each stride object. - update: removed that for now.
class Stride:
    def __init__(self, df_walk_data, start_idx, stop_idx):
        self.start_idx = start_idx
        self.stop_idx = stop_idx
        self.df_walk_data = df_walk_data
#         self.accel_z_smooth = accel_z_smooth

        total_disp, integral1, integral2 = get_displacement_of_walk_segment (df_walk_data, start_idx, stop_idx)
        self.horizontal_length = abs(total_disp)
        self.time_duration = (df_walk_data.time[self.stop_idx] - df_walk_data.time[self.start_idx])/1000
        self.speed = self.horizontal_length/self.time_duration #m/s


def get_displacement_of_walk_segment(df_data, start_idx, end_idx):
    # first integration
    integral_0 = df_data.accel_z[start_idx:end_idx]

    # first integral
    cumsum = 0
    integral_1 = []
    for idx, elem in enumerate(integral_0):
        delta_time = (df_data.time[start_idx+idx+1] - df_data.time[start_idx+idx] ) / 1000
        cumsum += elem * delta_time
        integral_1.append(cumsum)
    
    # second integral
    cumsum = 0
    integral_2 = []
    for idx, elem in enumerate(integral_1):
        delta_time = (df_data.time[start_idx+idx+1] - df_data.time[start_idx+idx] ) / 1000
        cumsum += elem * delta_time
        integral_2.append(cumsum)
    
    # total_displacement, first_integral, second_integral
    return cumsum, integral_1, integral_2

# TODO Mkimmins : segment increased to 500!
# segment_length = 100
segment_length = 100
temp_segment_compute_limit_start = 0
temp_segment_compute_limit_end = 400
# temp_segment_compute_limit_end = len(readings)
total_walk = []
# strides = []

# Algorithm Constants
start_quiet_noise_idx = 0
end_quiet_noise_idx = 100 # 1 second sigma calibration
STRIDE_TIMEOUT = 500
num_of_stddev = 3
gaussian_filter_sigma = 5


The third worker logic. this is where we do the calculations. 
total_data -> process_calculate_strides -> total_strides
This stores each strides to total_strides. Might cut this out as non-worker just simple cell you run for ease

In [8]:

def process_calculate_strides(total_data, total_strides):
    print("starting calculate_strides...")
    while True:
        try:
            walk_segment = total_data.get()
            print ("processing one segment chunk")
#             print("the segment: ", walk_segment)
            for k in walk_segment:total_walk.append(k) 
            df_walk = pd.DataFrame(total_walk)
            df_walk.time = df_walk.time - df_walk.time[0] # reset time from start of stream
            # Filter out accel_z
            accel_z_smooth = gaussian_filter1d(df_walk.accel_z, gaussian_filter_sigma)
            # Define Baseline Noise

            accel_z_std = np.std(df_walk.accel_z[start_quiet_noise_idx:end_quiet_noise_idx], axis=0)
            accel_z_mean = np.mean(df_walk.accel_z[start_quiet_noise_idx:end_quiet_noise_idx])
            upper_limit = accel_z_mean + num_of_stddev * accel_z_std
            lower_limit = accel_z_mean - num_of_stddev * accel_z_std

            # Determining Stride
            # Stride Class with auto-compute all chracterisitcs
            stride_under = False
            stride_over = False
            start_stride_idx, end_stride_idx = None, None
            # Determining Stride
            # Stride Class with auto-compute all chracterisitcs
            stride_under = False
            stride_over = False
            start_stride_idx, end_stride_idx = None, None

            for idx, data in enumerate(accel_z_smooth):
                # timeout
                if start_stride_idx and idx-start_stride_idx > STRIDE_TIMEOUT:
                    start_stride_idx = None
                    stride_under = False
                    stride_over = False
                if (data < lower_limit) and (not stride_under and not stride_over):
                    # Heal off the ground
                    stride_under = True
                    start_stride_idx = idx
                elif (data > upper_limit) and (stride_under and not stride_over):
                    stride_under = False
                    stride_over = True
                elif (data < upper_limit) and (not stride_under and stride_over):
                    stride_under = False
                    stride_over = False   
                    end_stride_idx = idx
                    curr_stride = Stride (df_walk, start_stride_idx, end_stride_idx)
                    print("end of reading for one stride.")
                    total_strides.put(curr_stride)
        except:
            print_exc()
            print("exception in stride processor worker")


The second worker.
It takes usb reads and chunks by segment_length, saves to total_data queue
usb_reading_queue -> stride_processor_worker -> total_data

In [9]:


#worker to process one stride boundary, sort data into buckets
def stride_processor_worker(total_data, raw_stream):
    this_segment = []
    print("stride_processor starting...")
    while True:
        #Infinitely 1. appends to this_segment, then when boundary is met,
        #appends that stride to total_data, reinitializes this_segment
        #and loops forever.
        try:
            row = raw_stream.get()
            print ("decoded: ", row)
            if len(this_segment) > segment_length:
                #put away the current stride in total data as one stride
                total_data.put(this_segment)
                print("finished writing this one stride, size: ", len(this_segment))
                #reinitialize the stride queue
                this_segment = []
            else:
                this_segment.append(row)
        except:
            print_exc()
            print("exception in stride processor worker")
            
    

In [10]:
# overall structure:
# total_data = [stride1, stride2,,,,,,stride500]
# stride1 = [{r1}, {r2}, {r3}]

In [11]:
######### start 'main' ########

# treat this as main of code: when all the runtime things happen

Set up multithreading contexts, and initialize global queue

In [12]:
pool = mp.Pool()
manager = mp.Manager()
usb_reading_queue = mp.SimpleQueue() # This queue holds all individual readings that are parsed.
total_data = mp.SimpleQueue() #This queue holds all chunked readings that are emptied from reading queue
total_strides = mp.SimpleQueue() #This queue drains total_data and chunks into strides with Prathik's computation



Process ForkPoolWorker-8:
Process ForkPoolWorker-7:
Process ForkPoolWorker-3:
Process ForkPoolWorker-2:
Process ForkPoolWorker-6:
Process ForkPoolWorker-1:
Process ForkPoolWorker-5:
Process ForkPoolWorker-4:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.9/site-packages/multiprocess/proc

#  set up worker threads

In [13]:
stride_process = mp.Process(target=stride_processor_worker, args=[total_data, usb_reading_queue])


In [14]:
arduino_proc = mp.Process(target=arduino_worker, args=[usb_reading_queue])


In [15]:
strides_compute_proc = mp.Process(target=process_calculate_strides, args=[total_data, total_strides])

# start threads

In [16]:
arduino_proc.start()

worker starting...
got b'time=701497,accel_x= -0.06,accel_y= -0.05,accel_z= -0.16,gyro_x= 0.00,gyro_y= 0.01,gyro_z= 0.01,mag_x= -18.25,mag_y= -0.19,mag_z= -8.56,orientation_x= 0.31,orientation_y= 31.00,orientation_z= -17.81\r\n'
time=701497,accel_x= -0.06,accel_y= -0.05,accel_z= -0.16,gyro_x= 0.00,gyro_y= 0.01,gyro_z= 0.01,mag_x= -18.25,mag_y= -0.19,mag_z= -8.56,orientation_x= 0.31,orientation_y= 31.00,orientation_z= -17.81

got b'time=701510,accel_x= -0.09,accel_y= -0.06,accel_z= -0.06,gyro_x= -0.02,gyro_y= 0.05,gyro_z= 0.02,mag_x= -18.25,mag_y= -0.19,mag_z= -8.56,orientation_x= 0.31,orientation_y= 31.00,orientation_z= -17.75\r\n'
time=701510,accel_x= -0.09,accel_y= -0.06,accel_z= -0.06,gyro_x= -0.02,gyro_y= 0.05,gyro_z= 0.02,mag_x= -18.25,mag_y= -0.19,mag_z= -8.56,orientation_x= 0.31,orientation_y= 31.00,orientation_z= -17.75

got b'time=701520,accel_x= -0.09,accel_y= -0.12,accel_z= 0.13,gyro_x= -0.01,gyro_y= 0.02,gyro_z= 0.01,mag_x= -18.25,mag_y= -0.19,mag_z= -8.56,orientation_x= 0.


got b'time=701720,accel_x= 0.00,accel_y= -0.12,accel_z= 0.22,gyro_x= -0.00,gyro_y= 0.06,gyro_z= 0.01,mag_x= -17.50,mag_y= 0.06,mag_z= -9.00,orientation_x= 0.31,orientation_y= 30.94,orientation_z= -17.75\r\n'
time=701720,accel_x= 0.00,accel_y= -0.12,accel_z= 0.22,gyro_x= -0.00,gyro_y= 0.06,gyro_z= 0.01,mag_x= -17.50,mag_y= 0.06,mag_z= -9.00,orientation_x= 0.31,orientation_y= 30.94,orientation_z= -17.75

got b'time=701730,accel_x= -0.10,accel_y= -0.05,accel_z= 0.04,gyro_x= 0.03,gyro_y= -0.03,gyro_z= 0.00,mag_x= -17.50,mag_y= 0.88,mag_z= -8.25,orientation_x= 0.31,orientation_y= 31.00,orientation_z= -17.75\r\n'
time=701730,accel_x= -0.10,accel_y= -0.05,accel_z= 0.04,gyro_x= 0.03,gyro_y= -0.03,gyro_z= 0.00,mag_x= -17.50,mag_y= 0.88,mag_z= -8.25,orientation_x= 0.31,orientation_y= 31.00,orientation_z= -17.75

got b'time=701742,accel_x= 0.04,accel_y= -0.12,accel_z= -0.18,gyro_x= 0.00,gyro_y= -0.05,gyro_z= -0.02,mag_x= -17.50,mag_y= 0.88,mag_z= -8.25,orientation_x= 0.31,orientation_y= 31.00,or

got b'time=701940,accel_x= -0.09,accel_y= -0.06,accel_z= 0.04,gyro_x= 0.00,gyro_y= -0.01,gyro_z= -0.03,mag_x= -17.50,mag_y= -0.19,mag_z= -8.25,orientation_x= 0.37,orientation_y= 31.00,orientation_z= -17.75\r\n'
time=701940,accel_x= -0.09,accel_y= -0.06,accel_z= 0.04,gyro_x= 0.00,gyro_y= -0.01,gyro_z= -0.03,mag_x= -17.50,mag_y= -0.19,mag_z= -8.25,orientation_x= 0.37,orientation_y= 31.00,orientation_z= -17.75

got b'time=701952,accel_x= -0.11,accel_y= -0.05,accel_z= -0.08,gyro_x= 0.01,gyro_y= -0.01,gyro_z= 0.01,mag_x= -17.50,mag_y= -0.19,mag_z= -8.25,orientation_x= 0.37,orientation_y= 31.00,orientation_z= -17.75\r\n'
time=701952,accel_x= -0.11,accel_y= -0.05,accel_z= -0.08,gyro_x= 0.01,gyro_y= -0.01,gyro_z= 0.01,mag_x= -17.50,mag_y= -0.19,mag_z= -8.25,orientation_x= 0.37,orientation_y= 31.00,orientation_z= -17.75

got b'time=701964,accel_x= -0.06,accel_y= -0.09,accel_z= 0.03,gyro_x= -0.01,gyro_y= 0.03,gyro_z= 0.02,mag_x= -17.50,mag_y= -0.19,mag_z= -8.25,orientation_x= 0.37,orientation_y=


got b'time=702172,accel_x= -0.13,accel_y= -0.06,accel_z= -0.04,gyro_x= 0.01,gyro_y= 0.03,gyro_z= -0.00,mag_x= -18.75,mag_y= 0.50,mag_z= -8.56,orientation_x= 0.31,orientation_y= 30.94,orientation_z= -17.75\r\n'
time=702172,accel_x= -0.13,accel_y= -0.06,accel_z= -0.04,gyro_x= 0.01,gyro_y= 0.03,gyro_z= -0.00,mag_x= -18.75,mag_y= 0.50,mag_z= -8.56,orientation_x= 0.31,orientation_y= 30.94,orientation_z= -17.75

got b'time=702182,accel_x= -0.04,accel_y= -0.06,accel_z= -0.10,gyro_x= -0.01,gyro_y= -0.03,gyro_z= 0.01,mag_x= -17.56,mag_y= 0.50,mag_z= -8.56,orientation_x= 0.31,orientation_y= 31.00,orientation_z= -17.75\r\n'
time=702182,accel_x= -0.04,accel_y= -0.06,accel_z= -0.10,gyro_x= -0.01,gyro_y= -0.03,gyro_z= 0.01,mag_x= -17.56,mag_y= 0.50,mag_z= -8.56,orientation_x= 0.31,orientation_y= 31.00,orientation_z= -17.75

got b'time=702194,accel_x= -0.01,accel_y= -0.09,accel_z= -0.06,gyro_x= -0.01,gyro_y= -0.02,gyro_z= -0.00,mag_x= -17.56,mag_y= 0.50,mag_z= -8.56,orientation_x= 0.31,orientation_y


got b'time=702397,accel_x= -0.04,accel_y= -0.05,accel_z= -0.01,gyro_x= -0.01,gyro_y= -0.02,gyro_z= -0.04,mag_x= -18.37,mag_y= 0.06,mag_z= -8.56,orientation_x= 0.31,orientation_y= 31.00,orientation_z= -17.75\r\n'
time=702397,accel_x= -0.04,accel_y= -0.05,accel_z= -0.01,gyro_x= -0.01,gyro_y= -0.02,gyro_z= -0.04,mag_x= -18.37,mag_y= 0.06,mag_z= -8.56,orientation_x= 0.31,orientation_y= 31.00,orientation_z= -17.75

got b'time=702410,accel_x= -0.06,accel_y= -0.03,accel_z= -0.08,gyro_x= 0.01,gyro_y= -0.01,gyro_z= -0.02,mag_x= -18.37,mag_y= 0.06,mag_z= -8.56,orientation_x= 0.37,orientation_y= 31.00,orientation_z= -17.75\r\n'
time=702410,accel_x= -0.06,accel_y= -0.03,accel_z= -0.08,gyro_x= 0.01,gyro_y= -0.01,gyro_z= -0.02,mag_x= -18.37,mag_y= 0.06,mag_z= -8.56,orientation_x= 0.37,orientation_y= 31.00,orientation_z= -17.75

got b'time=702420,accel_x= -0.03,accel_y= -0.05,accel_z= -0.09,gyro_x= 0.01,gyro_y= -0.00,gyro_z= 0.03,mag_x= -18.37,mag_y= 0.06,mag_z= -8.56,orientation_x= 0.37,orientation

...


got b'time=708415,accel_x= 0.00,accel_y= -0.09,accel_z= -0.04,gyro_x= 0.00,gyro_y= 0.01,gyro_z= 0.02,mag_x= -18.00,mag_y= -0.19,mag_z= -8.25,orientation_x= 0.25,orientation_y= 31.00,orientation_z= -17.81\r\n'
time=708415,accel_x= 0.00,accel_y= -0.09,accel_z= -0.04,gyro_x= 0.00,gyro_y= 0.01,gyro_z= 0.02,mag_x= -18.00,mag_y= -0.19,mag_z= -8.25,orientation_x= 0.25,orientation_y= 31.00,orientation_z= -17.81

got b'time=708427,accel_x= -0.01,accel_y= -0.10,accel_z= -0.04,gyro_x= 0.01,gyro_y= -0.00,gyro_z= -0.01,mag_x= -18.00,mag_y= 0.50,mag_z= -9.00,orientation_x= 0.25,orientation_y= 31.00,orientation_z= -17.81\r\n'
time=708427,accel_x= -0.01,accel_y= -0.10,accel_z= -0.04,gyro_x= 0.01,gyro_y= -0.00,gyro_z= -0.01,mag_x= -18.00,mag_y= 0.50,mag_z= -9.00,orientation_x= 0.25,orientation_y= 31.00,orientation_z= -17.81

got b'time=708439,accel_x= -0.04,accel_y= -0.08,accel_z= -0.01,gyro_x= -0.00,gyro_y= -0.01,gyro_z= -0.00,mag_x= -18.00,mag_y= 0.50,mag_z= -9.00,orientation_x= 0.31,orientation_y= 


got b'time=708654,accel_x= -0.13,accel_y= -0.06,accel_z= 0.00,gyro_x= 0.01,gyro_y= 0.01,gyro_z= -0.00,mag_x= -18.37,mag_y= 0.50,mag_z= -8.56,orientation_x= 0.31,orientation_y= 31.00,orientation_z= -17.81\r\n'
time=708654,accel_x= -0.13,accel_y= -0.06,accel_z= 0.00,gyro_x= 0.01,gyro_y= 0.01,gyro_z= -0.00,mag_x= -18.37,mag_y= 0.50,mag_z= -8.56,orientation_x= 0.31,orientation_y= 31.00,orientation_z= -17.81

got b'time=708666,accel_x= -0.04,accel_y= -0.15,accel_z= -0.07,gyro_x= 0.02,gyro_y= -0.00,gyro_z= 0.02,mag_x= -18.37,mag_y= 0.50,mag_z= -8.56,orientation_x= 0.25,orientation_y= 31.00,orientation_z= -17.81\r\n'
time=708666,accel_x= -0.04,accel_y= -0.15,accel_z= -0.07,gyro_x= 0.02,gyro_y= -0.00,gyro_z= 0.02,mag_x= -18.37,mag_y= 0.50,mag_z= -8.56,orientation_x= 0.25,orientation_y= 31.00,orientation_z= -17.81

got b'time=708676,accel_x= -0.06,accel_y= -0.12,accel_z= 0.06,gyro_x= -0.01,gyro_y= 0.03,gyro_z= 0.04,mag_x= -18.00,mag_y= 0.50,mag_z= -8.56,orientation_x= 0.25,orientation_y= 31.00

got b'time=708885,accel_x= 0.01,accel_y= -0.12,accel_z= 0.02,gyro_x= 0.00,gyro_y= -0.02,gyro_z= 0.02,mag_x= -18.00,mag_y= 0.50,mag_z= -7.87,orientation_x= 0.25,orientation_y= 31.00,orientation_z= -17.81\r\n'
time=708885,accel_x= 0.01,accel_y= -0.12,accel_z= 0.02,gyro_x= 0.00,gyro_y= -0.02,gyro_z= 0.02,mag_x= -18.00,mag_y= 0.50,mag_z= -7.87,orientation_x= 0.25,orientation_y= 31.00,orientation_z= -17.81

got b'time=708897,accel_x= -0.04,accel_y= -0.06,accel_z= -0.07,gyro_x= 0.00,gyro_y= -0.03,gyro_z= -0.02,mag_x= -18.00,mag_y= 0.50,mag_z= -7.87,orientation_x= 0.25,orientation_y= 31.00,orientation_z= -17.81\r\n'
time=708897,accel_x= -0.04,accel_y= -0.06,accel_z= -0.07,gyro_x= 0.00,gyro_y= -0.03,gyro_z= -0.02,mag_x= -18.00,mag_y= 0.50,mag_z= -7.87,orientation_x= 0.25,orientation_y= 31.00,orientation_z= -17.81

got b'time=708909,accel_x= -0.05,accel_y= -0.07,accel_z= -0.09,gyro_x= -0.01,gyro_y= -0.02,gyro_z= -0.03,mag_x= -18.00,mag_y= 0.50,mag_z= -7.87,orientation_x= 0.31,orientation_y= 31.


got b'time=709116,accel_x= 0.00,accel_y= -0.13,accel_z= -0.01,gyro_x= 0.02,gyro_y= 0.00,gyro_z= 0.03,mag_x= -17.56,mag_y= 0.50,mag_z= -8.56,orientation_x= 0.31,orientation_y= 31.00,orientation_z= -17.81\r\n'
time=709116,accel_x= 0.00,accel_y= -0.13,accel_z= -0.01,gyro_x= 0.02,gyro_y= 0.00,gyro_z= 0.03,mag_x= -17.56,mag_y= 0.50,mag_z= -8.56,orientation_x= 0.31,orientation_y= 31.00,orientation_z= -17.81

got b'time=709126,accel_x= -0.07,accel_y= -0.06,accel_z= 0.00,gyro_x= 0.02,gyro_y= 0.03,gyro_z= 0.04,mag_x= -18.00,mag_y= 0.50,mag_z= -7.50,orientation_x= 0.25,orientation_y= 31.00,orientation_z= -17.81\r\n'
time=709126,accel_x= -0.07,accel_y= -0.06,accel_z= 0.00,gyro_x= 0.02,gyro_y= 0.03,gyro_z= 0.04,mag_x= -18.00,mag_y= 0.50,mag_z= -7.50,orientation_x= 0.25,orientation_y= 31.00,orientation_z= -17.81

got b'time=709136,accel_x= -0.05,accel_y= -0.07,accel_z= -0.02,gyro_x= 0.01,gyro_y= 0.03,gyro_z= 0.01,mag_x= -18.00,mag_y= 0.50,mag_z= -7.50,orientation_x= 0.25,orientation_y= 31.00,orien

Process Process-11:
Traceback (most recent call last):
  File "<ipython-input-6-619db66a6851>", line 14, in arduino_worker
    queue.put(row)
  File "/usr/local/lib/python3.9/site-packages/multiprocess/queues.py", line 381, in put
    self._writer.send_bytes(obj)
  File "/usr/local/lib/python3.9/site-packages/multiprocess/connection.py", line 208, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/local/lib/python3.9/site-packages/multiprocess/connection.py", line 419, in _send_bytes
    self._send(header + buf)
  File "/usr/local/lib/python3.9/site-packages/multiprocess/connection.py", line 376, in _send
    n = write(self._handle, buf)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-6-619db66a6851>", line 16, in arduino_worker
    print_exc()
NameError: name 'print_exc' is not defined

During handling of the above exception, another exception occurred:

Traceback (m

In [17]:
stride_process.start()

stride_processor starting...
decoded:  {'time': 701497.0, 'accel_x': -0.06, 'accel_y': -0.05, 'accel_z': -0.16, 'gyro_x': 0.0, 'gyro_y': 0.01, 'gyro_z': 0.01, 'mag_x': -18.25, 'mag_y': -0.19, 'mag_z': -8.56, 'orientation_x': 0.31, 'orientation_y': 31.0}
decoded:  {'time': 701510.0, 'accel_x': -0.09, 'accel_y': -0.06, 'accel_z': -0.06, 'gyro_x': -0.02, 'gyro_y': 0.05, 'gyro_z': 0.02, 'mag_x': -18.25, 'mag_y': -0.19, 'mag_z': -8.56, 'orientation_x': 0.31, 'orientation_y': 31.0}
decoded:  {'time': 701520.0, 'accel_x': -0.09, 'accel_y': -0.12, 'accel_z': 0.13, 'gyro_x': -0.01, 'gyro_y': 0.02, 'gyro_z': 0.01, 'mag_x': -18.25, 'mag_y': -0.19, 'mag_z': -8.56, 'orientation_x': 0.31, 'orientation_y': 31.0}
decoded:  {'time': 701530.0, 'accel_x': -0.03, 'accel_y': -0.12, 'accel_z': -0.02, 'gyro_x': 0.01, 'gyro_y': -0.03, 'gyro_z': 0.01, 'mag_x': -17.87, 'mag_y': 0.06, 'mag_z': -9.75, 'orientation_x': 0.31, 'orientation_y': 31.0}
decoded:  {'time': 701542.0, 'accel_x': -0.09, 'accel_y': -0.06, 'a

decoded:  {'time': 701896.0, 'accel_x': -0.02, 'accel_y': -0.08, 'accel_z': -0.03, 'gyro_x': 0.0, 'gyro_y': 0.0, 'gyro_z': 0.02, 'mag_x': -18.25, 'mag_y': 0.06, 'mag_z': -8.56, 'orientation_x': 0.31, 'orientation_y': 31.0}
decoded:  {'time': 701908.0, 'accel_x': 0.04, 'accel_y': -0.09, 'accel_z': -0.07, 'gyro_x': -0.0, 'gyro_y': 0.0, 'gyro_z': 0.01, 'mag_x': -18.25, 'mag_y': 0.06, 'mag_z': -8.56, 'orientation_x': 0.31, 'orientation_y': 31.0}
decoded:  {'time': 701918.0, 'accel_x': 0.0, 'accel_y': -0.11, 'accel_z': -0.09, 'gyro_x': 0.0, 'gyro_y': -0.01, 'gyro_z': -0.02, 'mag_x': -18.25, 'mag_y': 0.06, 'mag_z': -8.56, 'orientation_x': 0.31, 'orientation_y': 31.0}
decoded:  {'time': 701928.0, 'accel_x': -0.06, 'accel_y': -0.04, 'accel_z': -0.04, 'gyro_x': -0.01, 'gyro_y': -0.02, 'gyro_z': -0.04, 'mag_x': -17.5, 'mag_y': -0.19, 'mag_z': -8.25, 'orientation_x': 0.37, 'orientation_y': 31.0}
decoded:  {'time': 701940.0, 'accel_x': -0.09, 'accel_y': -0.06, 'accel_z': 0.04, 'gyro_x': 0.0, 'gyro

decoded:  {'time': 702308.0, 'accel_x': -0.1, 'accel_y': -0.03, 'accel_z': -0.02, 'gyro_x': 0.0, 'gyro_y': -0.0, 'gyro_z': -0.03, 'mag_x': -18.37, 'mag_y': -0.56, 'mag_z': -9.0, 'orientation_x': 0.37, 'orientation_y': 31.0}
decoded:  {'time': 702318.0, 'accel_x': -0.13, 'accel_y': -0.02, 'accel_z': 0.0, 'gyro_x': 0.01, 'gyro_y': 0.01, 'gyro_z': 0.01, 'mag_x': -18.37, 'mag_y': -0.56, 'mag_z': -9.0, 'orientation_x': 0.37, 'orientation_y': 31.0}
decoded:  {'time': 702328.0, 'accel_x': -0.04, 'accel_y': -0.07, 'accel_z': -0.12, 'gyro_x': 0.01, 'gyro_y': 0.02, 'gyro_z': 0.03, 'mag_x': -18.37, 'mag_y': 0.88, 'mag_z': -9.0, 'orientation_x': 0.31, 'orientation_y': 31.0}
decoded:  {'time': 702340.0, 'accel_x': -0.1, 'accel_y': -0.04, 'accel_z': 0.06, 'gyro_x': 0.0, 'gyro_y': 0.03, 'gyro_z': 0.03, 'mag_x': -18.37, 'mag_y': 0.88, 'mag_z': -9.0, 'orientation_x': 0.31, 'orientation_y': 31.0}
decoded:  {'time': 702353.0, 'accel_x': -0.04, 'accel_y': -0.11, 'accel_z': -0.06, 'gyro_x': 0.01, 'gyro_y':

decoded:  {'time': 702712.0, 'accel_x': -0.11, 'accel_y': -0.05, 'accel_z': -0.02, 'gyro_x': 0.01, 'gyro_y': 0.01, 'gyro_z': 0.01, 'mag_x': -17.5, 'mag_y': -0.56, 'mag_z': -8.25, 'orientation_x': 0.31, 'orientation_y': 31.0}
decoded:  {'time': 702724.0, 'accel_x': -0.1, 'accel_y': -0.06, 'accel_z': 0.0, 'gyro_x': 0.01, 'gyro_y': 0.02, 'gyro_z': 0.03, 'mag_x': -18.69, 'mag_y': -0.19, 'mag_z': -7.87, 'orientation_x': 0.31, 'orientation_y': 31.0}
decoded:  {'time': 702737.0, 'accel_x': 0.03, 'accel_y': -0.1, 'accel_z': 0.14, 'gyro_x': -0.01, 'gyro_y': 0.01, 'gyro_z': 0.0, 'mag_x': -18.69, 'mag_y': -0.19, 'mag_z': -7.87, 'orientation_x': 0.31, 'orientation_y': 30.94}
decoded:  {'time': 702750.0, 'accel_x': -0.07, 'accel_y': -0.06, 'accel_z': -0.03, 'gyro_x': 0.02, 'gyro_y': -0.01, 'gyro_z': -0.01, 'mag_x': -18.69, 'mag_y': -0.19, 'mag_z': -7.87, 'orientation_x': 0.31, 'orientation_y': 31.0}
decoded:  {'time': 702763.0, 'accel_x': 0.13, 'accel_y': -0.17, 'accel_z': -0.17, 'gyro_x': 0.0, 'gy

decoded:  {'time': 703127.0, 'accel_x': -0.01, 'accel_y': -0.12, 'accel_z': -0.09, 'gyro_x': 0.0, 'gyro_y': -0.03, 'gyro_z': 0.0, 'mag_x': -18.0, 'mag_y': 0.5, 'mag_z': -8.56, 'orientation_x': 0.31, 'orientation_y': 31.0}
decoded:  {'time': 703140.0, 'accel_x': 0.0, 'accel_y': -0.1, 'accel_z': -0.06, 'gyro_x': -0.01, 'gyro_y': -0.01, 'gyro_z': -0.01, 'mag_x': -18.0, 'mag_y': 0.5, 'mag_z': -8.56, 'orientation_x': 0.31, 'orientation_y': 31.0}
decoded:  {'time': 703153.0, 'accel_x': -0.02, 'accel_y': -0.08, 'accel_z': 0.0, 'gyro_x': -0.01, 'gyro_y': -0.01, 'gyro_z': -0.02, 'mag_x': -18.0, 'mag_y': 0.5, 'mag_z': -8.56, 'orientation_x': 0.37, 'orientation_y': 31.0}
decoded:  {'time': 703165.0, 'accel_x': -0.08, 'accel_y': -0.06, 'accel_z': 0.0, 'gyro_x': 0.0, 'gyro_y': -0.0, 'gyro_z': -0.03, 'mag_x': -18.0, 'mag_y': 0.5, 'mag_z': -8.56, 'orientation_x': 0.37, 'orientation_y': 31.0}
decoded:  {'time': 703175.0, 'accel_x': -0.03, 'accel_y': -0.06, 'accel_z': -0.06, 'gyro_x': 0.0, 'gyro_y': -0

...

decoded:  {'time': 704801.0, 'accel_x': 0.03, 'accel_y': -0.23, 'accel_z': 0.11, 'gyro_x': -0.02, 'gyro_y': 0.06, 'gyro_z': 0.04, 'mag_x': -17.19, 'mag_y': 0.88, 'mag_z': -9.0, 'orientation_x': 0.37, 'orientation_y': 31.0}
decoded:  {'time': 704814.0, 'accel_x': 0.16, 'accel_y': -0.22, 'accel_z': -0.33, 'gyro_x': 0.07, 'gyro_y': 0.03, 'gyro_z': 0.05, 'mag_x': -17.19, 'mag_y': 0.88, 'mag_z': -9.0, 'orientation_x': 0.31, 'orientation_y': 30.94}
decoded:  {'time': 704824.0, 'accel_x': -0.2, 'accel_y': 0.01, 'accel_z': 0.14, 'gyro_x': -0.03, 'gyro_y': 0.05, 'gyro_z': -0.04, 'mag_x': -18.37, 'mag_y': -0.19, 'mag_z': -7.06, 'orientation_x': 0.31, 'orientation_y': 30.94}
decoded:  {'time': 704834.0, 'accel_x': -0.09, 'accel_y': -0.06, 'accel_z': 0.13, 'gyro_x': 0.02, 'gyro_y': -0.03, 'gyro_z': 0.06, 'mag_x': -18.37, 'mag_y': -0.19, 'mag_z': -7.06, 'orientation_x': 0.31, 'orientation_y': 30.94}
decoded:  {'time': 704846.0, 'accel_x': -0.03, 'accel_y': -0.08, 'accel_z': 0.14, 'gyro_x': 0.01, 'g

decoded:  {'time': 705205.0, 'accel_x': 0.0, 'accel_y': -0.09, 'accel_z': -0.04, 'gyro_x': -0.0, 'gyro_y': -0.02, 'gyro_z': 0.01, 'mag_x': -18.0, 'mag_y': -0.19, 'mag_z': -8.56, 'orientation_x': 0.31, 'orientation_y': 30.94}
decoded:  {'time': 705216.0, 'accel_x': 0.0, 'accel_y': -0.08, 'accel_z': -0.04, 'gyro_x': -0.0, 'gyro_y': -0.03, 'gyro_z': -0.04, 'mag_x': -18.0, 'mag_y': -0.19, 'mag_z': -8.56, 'orientation_x': 0.31, 'orientation_y': 31.0}
decoded:  {'time': 705226.0, 'accel_x': -0.1, 'accel_y': 0.02, 'accel_z': -0.01, 'gyro_x': -0.02, 'gyro_y': -0.04, 'gyro_z': -0.06, 'mag_x': -17.56, 'mag_y': -0.19, 'mag_z': -9.0, 'orientation_x': 0.37, 'orientation_y': 31.0}
decoded:  {'time': 705237.0, 'accel_x': 0.09, 'accel_y': 0.01, 'accel_z': 0.07, 'gyro_x': -0.01, 'gyro_y': -0.01, 'gyro_z': -0.01, 'mag_x': -17.56, 'mag_y': -0.19, 'mag_z': -9.0, 'orientation_x': 0.37, 'orientation_y': 31.0}
decoded:  {'time': 705250.0, 'accel_x': -0.1, 'accel_y': 0.0, 'accel_z': 0.08, 'gyro_x': 0.03, 'gyr

decoded:  {'time': 705616.0, 'accel_x': 0.0, 'accel_y': -0.08, 'accel_z': -0.04, 'gyro_x': -0.01, 'gyro_y': -0.0, 'gyro_z': 0.0, 'mag_x': -17.56, 'mag_y': 0.88, 'mag_z': -8.25, 'orientation_x': 0.37, 'orientation_y': 31.0}
decoded:  {'time': 705628.0, 'accel_x': -0.02, 'accel_y': -0.07, 'accel_z': 0.02, 'gyro_x': -0.01, 'gyro_y': 0.01, 'gyro_z': -0.01, 'mag_x': -17.25, 'mag_y': 0.5, 'mag_z': -9.0, 'orientation_x': 0.37, 'orientation_y': 31.0}
decoded:  {'time': 705641.0, 'accel_x': -0.06, 'accel_y': -0.04, 'accel_z': -0.05, 'gyro_x': 0.02, 'gyro_y': 0.01, 'gyro_z': 0.0, 'mag_x': -17.25, 'mag_y': 0.5, 'mag_z': -9.0, 'orientation_x': 0.37, 'orientation_y': 31.0}
decoded:  {'time': 705654.0, 'accel_x': -0.08, 'accel_y': -0.05, 'accel_z': -0.14, 'gyro_x': 0.01, 'gyro_y': 0.01, 'gyro_z': 0.05, 'mag_x': -17.25, 'mag_y': 0.5, 'mag_z': -9.0, 'orientation_x': 0.31, 'orientation_y': 31.0}
decoded:  {'time': 705666.0, 'accel_x': -0.06, 'accel_y': -0.11, 'accel_z': -0.06, 'gyro_x': 0.0, 'gyro_y': 

decoded:  {'time': 706044.0, 'accel_x': 0.0, 'accel_y': -0.1, 'accel_z': 0.0, 'gyro_x': 0.01, 'gyro_y': 0.04, 'gyro_z': 0.02, 'mag_x': -18.0, 'mag_y': 0.06, 'mag_z': -8.56, 'orientation_x': 0.31, 'orientation_y': 30.94}
decoded:  {'time': 706056.0, 'accel_x': -0.07, 'accel_y': -0.06, 'accel_z': 0.01, 'gyro_x': 0.0, 'gyro_y': 0.03, 'gyro_z': 0.0, 'mag_x': -18.0, 'mag_y': 0.06, 'mag_z': -8.56, 'orientation_x': 0.31, 'orientation_y': 30.94}
decoded:  {'time': 706066.0, 'accel_x': -0.07, 'accel_y': -0.06, 'accel_z': -0.02, 'gyro_x': 0.0, 'gyro_y': -0.03, 'gyro_z': -0.0, 'mag_x': -18.0, 'mag_y': 0.06, 'mag_z': -8.56, 'orientation_x': 0.31, 'orientation_y': 30.94}
decoded:  {'time': 706076.0, 'accel_x': -0.03, 'accel_y': -0.08, 'accel_z': -0.07, 'gyro_x': 0.01, 'gyro_y': -0.03, 'gyro_z': -0.02, 'mag_x': -18.37, 'mag_y': 0.5, 'mag_z': -9.5, 'orientation_x': 0.31, 'orientation_y': 30.94}
decoded:  {'time': 706087.0, 'accel_x': -0.11, 'accel_y': -0.05, 'accel_z': 0.0, 'gyro_x': -0.01, 'gyro_y':

Process Process-10:
Traceback (most recent call last):
  File "<ipython-input-9-766de4783058>", line 14, in stride_processor_worker
    total_data.put(this_segment)
  File "/usr/local/lib/python3.9/site-packages/multiprocess/queues.py", line 381, in put
    self._writer.send_bytes(obj)
  File "/usr/local/lib/python3.9/site-packages/multiprocess/connection.py", line 208, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/local/lib/python3.9/site-packages/multiprocess/connection.py", line 413, in _send_bytes
    self._send(buf)
  File "/usr/local/lib/python3.9/site-packages/multiprocess/connection.py", line 376, in _send
    n = write(self._handle, buf)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.9/site-packages/multiprocess/process.py", line 108, 

In [ ]:
strides_compute_proc.start()

ONLY run this cell when you want to kill the processes.
if you want to inspect the intermediary queues, just kill the processes prior to that
so that the previous processes can put into queue without it being drained.

# kill threads

In [ ]:
# arduino_proc.join()
# stride_process.join()
# strides_compute_proc.join()
arduino_proc.terminate()
stride_process.terminate()
strides_compute_proc.terminate()

# SCRATCHWORK BEGINS HERE

In [ ]:

strides = []
while True:
    try:
        walk_segment = total_data.get()
        print ("processing one segment chunk")
        for k in walk_segment:total_walk.append(k) 
        df_walk = pd.DataFrame(total_walk)
        df_walk.time = df_walk.time - df_walk.time[0] # reset time from start of stream
        # Filter out accel_z
        accel_z_smooth = gaussian_filter1d(df_walk.accel_z, gaussian_filter_sigma)
        # Define Baseline Noise

        accel_z_std = np.std(df_walk.accel_z[start_quiet_noise_idx:end_quiet_noise_idx], axis=0)
        accel_z_mean = np.mean(df_walk.accel_z[start_quiet_noise_idx:end_quiet_noise_idx])
        upper_limit = accel_z_mean + num_of_stddev * accel_z_std
        lower_limit = accel_z_mean - num_of_stddev * accel_z_std

        # Determining Stride
        # Stride Class with auto-compute all chracterisitcs
        stride_under = False
        stride_over = False
        start_stride_idx, end_stride_idx = None, None
        # Determining Stride
        # Stride Class with auto-compute all chracterisitcs
        stride_under = False
        stride_over = False
        start_stride_idx, end_stride_idx = None, None

        for idx, data in enumerate(accel_z_smooth):
            # timeout
            if start_stride_idx and idx-start_stride_idx > STRIDE_TIMEOUT:
                start_stride_idx = None
                stride_under = False
                stride_over = False
            if (data < lower_limit) and (not stride_under and not stride_over):
                # Heal off the ground
                stride_under = True
                start_stride_idx = idx
            elif (data > upper_limit) and (stride_under and not stride_over):
                stride_under = False
                stride_over = True
            elif (data < upper_limit) and (not stride_under and stride_over):
                stride_under = False
                stride_over = False   
                end_stride_idx = idx
                curr_stride = Stride (df_walk, start_stride_idx, end_stride_idx)
                print("end of reading for one stride.")
                total_strides.put(curr_stride)

        fig = plt.figure(figsize=(14, 4), dpi=80)
        plt.plot(df_walk.time,df_walk.accel_z)
        plt.plot(df_walk.time,accel_z_smooth)
        plt.axhline(y=upper_limit, color='r', linestyle='-')
        plt.axhline(y=lower_limit, color='r', linestyle='-')
        plt.show()
        print ("Number of Strides: ", len(strides))
        for idx, stride in enumerate(strides):
            print ("Stride ", idx, " :") 
            print ("Stride Start Time: ", df_walk.time[stride.start_idx]/1000)
            print ("Stride End Time: ", df_walk.time[stride.stop_idx]/1000)
            print ("Stride Length (m):", stride.horizontal_length)
            print ("Stride Time Duration (sec): ", stride.time_duration)
            print ("Stride Time Duration (m/s): ", stride.speed)

        time.sleep(1)
    except:
        print("exception in plotting loop")

processing one segment chunk
exception in plotting loop
processing one segment chunk


In [ ]:
total_strides

In [ ]:
total_strides.empty()

In [ ]:
one_stride = total_strides.get()

In [ ]:
one_stride

In [ ]:
df_walk = one_stride.df_walk_data
accel_z_smooth = one_stride.accel_z_smooth

In [ ]:
fig = plt.figure(figsize=(14, 4), dpi=80)

In [ ]:
plt.plot(df_walk.time,df_walk.accel_z)
plt.plot(df_walk.time,accel_z_smooth)
plt.axhline(y=upper_limit, color='r', linestyle='-')
plt.axhline(y=lower_limit, color='r', linestyle='-')


In [ ]:

while True:
    try:
        plt.plot(df_walk.time,df_walk.accel_z)
        plt.plot(df_walk.time,accel_z_smooth)
        plt.axhline(y=upper_limit, color='r', linestyle='-')
        plt.axhline(y=lower_limit, color='r', linestyle='-')
#         plt.show()
        plt.draw()
        print ("Number of Strides: ", len(strides))
        for idx, stride in enumerate(strides):
            print ("Stride ", idx, " :") 
            print ("Stride Start Time: ", df_walk.time[stride.start_idx]/1000)
            print ("Stride End Time: ", df_walk.time[stride.stop_idx]/1000)
            print ("Stride Length (m):", stride.horizontal_length)
            print ("Stride Time Duration (sec): ", stride.time_duration)
            print ("Stride Time Duration (m/s): ", stride.speed)

        time.sleep(2)
    except:
        print("exception in plotting loop")


In [ ]:
total_data.empty()

In [ ]:
total_strides.empty()

In [ ]:

                    
                    
                    # print ((df_walk.shape))
            fig = plt.figure(figsize=(14, 4), dpi=80)
            plt.plot(df_walk.time,df_walk.accel_z)
            plt.plot(df_walk.time,accel_z_smooth)
            plt.axhline(y=upper_limit, color='r', linestyle='-')
            plt.axhline(y=lower_limit, color='r', linestyle='-')
            plt.show()
            print ("Number of Strides: ", len(strides))
            for idx, stride in enumerate(strides):
                print ("Stride ", idx, " :") 
                print ("Stride Start Time: ", df_walk.time[stride.start_idx]/1000)
                print ("Stride End Time: ", df_walk.time[stride.stop_idx]/1000)
                print ("Stride Length (m):", stride.horizontal_length)
                print ("Stride Time Duration (sec): ", stride.time_duration)
                print ("Stride Time Duration (m/s): ", stride.speed)

            time.sleep(1)

In [ ]:
total_data

In [ ]:
total_data.empty()

In [ ]:
stride_1 = total_data.get()

In [ ]:
stride_1

In [ ]:
# scratch

In [ ]:
walk_segment = total_data.get()
print("the segment: ", walk_segment)
for k in walk_segment:total_walk.append(k) 
df_walk = pd.DataFrame(total_walk)
df_walk.time = df_walk.time - df_walk.time[0] # reset time from start of stream
# Filter out accel_z
accel_z_smooth = gaussian_filter1d(df_walk.accel_z, gaussian_filter_sigma)
# Define Baseline Noise

accel_z_std = np.std(df_walk.accel_z[start_quiet_noise_idx:end_quiet_noise_idx], axis=0)
accel_z_mean = np.mean(df_walk.accel_z[start_quiet_noise_idx:end_quiet_noise_idx])
upper_limit = accel_z_mean + num_of_stddev * accel_z_std
lower_limit = accel_z_mean - num_of_stddev * accel_z_std

# Determining Stride
# Stride Class with auto-compute all chracterisitcs
stride_under = False
stride_over = False
start_stride_idx, end_stride_idx = None, None


In [ ]:

# Determining Stride
# Stride Class with auto-compute all chracterisitcs
stride_under = False
stride_over = False
start_stride_idx, end_stride_idx = None, None

for idx, data in enumerate(accel_z_smooth):
    # timeout
    if start_stride_idx and idx-start_stride_idx > STRIDE_TIMEOUT:
        start_stride_idx = None
        stride_under = False
        stride_over = False
    if (data < lower_limit) and (not stride_under and not stride_over):
        # Heal off the ground
        stride_under = True
        start_stride_idx = idx
    elif (data > upper_limit) and (stride_under and not stride_over):
        stride_under = False
        stride_over = True
    elif (data < upper_limit) and (not stride_under and stride_over):
        stride_under = False
        stride_over = False   
        end_stride_idx = idx
        curr_stride = Stride (df_walk, start_stride_idx, end_stride_idx)
        strides.append(curr_stride)



In [ ]:
for idx, data in enumerate(accel_z_smooth):
    print("idx: ", idx)
    print("data: ", data)

In [ ]:
strides

In [ ]:
decoded_bytes = "time=38758,accel_x= 0.03,accel_y= 0.03,accel_z= -0.39,gyro_x= 0.00,gyro_y= -0.00,gyro_z= -0.00,mag_x= 4.50,mag_y= 3.06,mag_z= -10.50,orientation_x= 359.94,orientation_y= 4.31,orientation_z= 0.44"

In [ ]:
row = process_serial_out(decoded_bytes)
print(decoded_bytes)

In [ ]:
ascii_string = decoded_bytes
ascii_string += ','
findall = re.findall(r'(?P<var>.*?)=(?P<out>.*?),', ascii_string)
extracted = {k.strip().lower(): float(v.strip()) for k,v in findall}


In [ ]:
extracted